In [1]:
import pickle
from copy import deepcopy
import time

# data prep and model-tuning
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# types of models we'll fit
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.multioutput import RegressorChain

/storage/miniconda3/envs/forest_mapping/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats

In [3]:
from warnings import simplefilter
simplefilter(action='ignore', category=UserWarning)

In [4]:
sns.set_style('darkgrid')
%matplotlib inline

## Our Data

In [5]:
PLOT_DATA = '../data/processed/plot_features.csv'
KEEP_PLOT_COLS = ['uuid', 'lat', 'lon', 'ecoregion3', 'agency', 'distance_to_water_m', 'plot_size_ac', 'meas_yr']
plot_data = pd.read_csv(PLOT_DATA)[KEEP_PLOT_COLS]
plot_data.columns = [col.upper() for col in plot_data.columns]
plot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5089 entries, 0 to 5088
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UUID                 5089 non-null   object 
 1   LAT                  5089 non-null   float64
 2   LON                  5089 non-null   float64
 3   ECOREGION3           5089 non-null   object 
 4   AGENCY               5089 non-null   object 
 5   DISTANCE_TO_WATER_M  5089 non-null   float64
 6   PLOT_SIZE_AC         5089 non-null   float64
 7   MEAS_YR              5089 non-null   int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 318.2+ KB


In [7]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

LIDAR_DATA = '../data/processed/lidar_features.csv'
lidar_data = pd.read_csv(LIDAR_DATA)
USE_LIDAR_COLS = ['strat0_return-proportion', 'strat1_return-proportion', 
                  'strat2_return-proportion', 'strat3_return-proportion', 
                  'strat4_return-proportion', 'strat5_return-proportion', 
                  'height_05-percentile',  'height_25-percentile', 
                  'height_50-percentile', 'height_75-percentile',
                  'height_95_percentile', 'cover', 
                  'potential_volume', 'stddev_height', 
                  'surface_volume', 'kurtosis', 'skewness',
                  'aspect', 'elevation', 'slope'
                 ]
lidar_data[USE_LIDAR_COLS] = lidar_data[USE_LIDAR_COLS].replace('-1.#IND00', np.nan).astype(float)
lidar_data = lidar_data[['uuid', 'lidar_year', 'lidar_acq'] + USE_LIDAR_COLS]
float_cols = [col for col in lidar_data.columns if lidar_data[col].dtype == 'float']

for col in lidar_data[float_cols].columns[np.any(lidar_data[float_cols].isna(),axis=0).values]:
    lidar_data[col] = imputer.fit_transform(lidar_data[col].values.reshape(-1, 1))
lidar_data.columns = [col.upper() for col in lidar_data.columns]
lidar_data = lidar_data.set_index('UUID') 

lidar_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5411 entries, 00027724 to fff7e1c3
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LIDAR_YEAR                5411 non-null   int64  
 1   LIDAR_ACQ                 5411 non-null   object 
 2   STRAT0_RETURN-PROPORTION  5411 non-null   float64
 3   STRAT1_RETURN-PROPORTION  5411 non-null   float64
 4   STRAT2_RETURN-PROPORTION  5411 non-null   float64
 5   STRAT3_RETURN-PROPORTION  5411 non-null   float64
 6   STRAT4_RETURN-PROPORTION  5411 non-null   float64
 7   STRAT5_RETURN-PROPORTION  5411 non-null   float64
 8   HEIGHT_05-PERCENTILE      5411 non-null   float64
 9   HEIGHT_25-PERCENTILE      5411 non-null   float64
 10  HEIGHT_50-PERCENTILE      5411 non-null   float64
 11  HEIGHT_75-PERCENTILE      5411 non-null   float64
 12  HEIGHT_95_PERCENTILE      5411 non-null   float64
 13  COVER                     5411 non-null   float64
 14  PO

In [8]:
INVENTORY = '../data/processed/inventory_features.csv'
inv_data = pd.read_csv(INVENTORY)
inv_data.columns = [col.upper() for col in inv_data.columns]
inv_data = inv_data.set_index('UUID')
inv_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49410 entries, ba510248 to c4f7f099
Data columns (total 76 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   YEAR                 49410 non-null  int64  
 1   TPA                  49410 non-null  int64  
 2   BA                   49410 non-null  int64  
 3   SDI                  49410 non-null  int64  
 4   CCF                  49410 non-null  int64  
 5   QMD                  49410 non-null  float64
 6   TCUFT                49410 non-null  int64  
 7   TOPHT                49410 non-null  int64  
 8   NUMBER_OF_STRATA     49410 non-null  int64  
 9   TOTAL_COVER          49410 non-null  int64  
 10  STRUCTURE_CLASS      49410 non-null  object 
 11  CANOPY_BASEHEIGHT    49410 non-null  int64  
 12  CANOPY_BULKDENSITY   49410 non-null  float64
 13  ABOVEGROUND_BIOMASS  49410 non-null  int64  
 14  ABOVEGROUND_CARBON   49410 non-null  int64  
 15  GS_TPA               49410 non-

## Filter out some of the training data
We can exclude some of the training data based on how far separated the inventory data (interpolated using FVS simulations) is from the year the lidar was collected. Similarly, we can screen out training examples that had relatively low density of lidar returns.

In [9]:
lidar_and_inv = lidar_data.merge(inv_data, how='outer', left_index=True, right_index=True)
lidar_and_inv = lidar_and_inv.reset_index()
lidar_and_inv = lidar_and_inv.loc[lidar_and_inv.UUID.isin(lidar_data.index.get_level_values(0))]
lidar_and_inv = lidar_and_inv.dropna(subset=['YEAR'])
lidar_and_inv['YEAR_DIFF'] = abs(lidar_and_inv['YEAR'] - lidar_and_inv['LIDAR_YEAR'])

In [10]:
lidar_and_inv = lidar_and_inv.loc[lidar_and_inv.groupby(by=['UUID', 'LIDAR_YEAR'])['YEAR_DIFF'].idxmin().astype(int)]
lidar_and_inv.index = lidar_and_inv.index.astype(int)
lidar_and_inv = lidar_and_inv.loc[lidar_and_inv.YEAR_DIFF <= 5]
lidar_and_inv['LIDAR_YEAR'] = lidar_and_inv['LIDAR_YEAR'].astype(int)
lidar_and_inv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4073 entries, 0 to 60804
Data columns (total 100 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   UUID                      4073 non-null   object 
 1   LIDAR_YEAR                4073 non-null   int64  
 2   LIDAR_ACQ                 4073 non-null   object 
 3   STRAT0_RETURN-PROPORTION  4073 non-null   float64
 4   STRAT1_RETURN-PROPORTION  4073 non-null   float64
 5   STRAT2_RETURN-PROPORTION  4073 non-null   float64
 6   STRAT3_RETURN-PROPORTION  4073 non-null   float64
 7   STRAT4_RETURN-PROPORTION  4073 non-null   float64
 8   STRAT5_RETURN-PROPORTION  4073 non-null   float64
 9   HEIGHT_05-PERCENTILE      4073 non-null   float64
 10  HEIGHT_25-PERCENTILE      4073 non-null   float64
 11  HEIGHT_50-PERCENTILE      4073 non-null   float64
 12  HEIGHT_75-PERCENTILE      4073 non-null   float64
 13  HEIGHT_95_PERCENTILE      4073 non-null   float64
 14  COVER 

In [11]:
df = lidar_and_inv.merge(plot_data, how='outer', left_on=['UUID'], right_on=['UUID']).dropna()
df['LIDAR_YEAR'] = df['LIDAR_YEAR'].astype(int)
print('{:,d} samples'.format(len(df)))
print('Columns:', df.columns.values)

4,073 samples
Columns: ['UUID' 'LIDAR_YEAR' 'LIDAR_ACQ' 'STRAT0_RETURN-PROPORTION'
 'STRAT1_RETURN-PROPORTION' 'STRAT2_RETURN-PROPORTION'
 'STRAT3_RETURN-PROPORTION' 'STRAT4_RETURN-PROPORTION'
 'STRAT5_RETURN-PROPORTION' 'HEIGHT_05-PERCENTILE' 'HEIGHT_25-PERCENTILE'
 'HEIGHT_50-PERCENTILE' 'HEIGHT_75-PERCENTILE' 'HEIGHT_95_PERCENTILE'
 'COVER' 'POTENTIAL_VOLUME' 'STDDEV_HEIGHT' 'SURFACE_VOLUME' 'KURTOSIS'
 'SKEWNESS' 'ASPECT' 'ELEVATION' 'SLOPE' 'YEAR' 'TPA' 'BA' 'SDI' 'CCF'
 'QMD' 'TCUFT' 'TOPHT' 'NUMBER_OF_STRATA' 'TOTAL_COVER' 'STRUCTURE_CLASS'
 'CANOPY_BASEHEIGHT' 'CANOPY_BULKDENSITY' 'ABOVEGROUND_BIOMASS'
 'ABOVEGROUND_CARBON' 'GS_TPA' 'AF' 'AS' 'BM' 'BO' 'CH' 'CW' 'DF' 'DG'
 'ES' 'GC' 'GF' 'IC' 'JP' 'KP' 'LO' 'LP' 'MA' 'MC' 'MH' 'NF' 'OH' 'OS'
 'OT' 'PC' 'PL' 'PP' 'PY' 'RA' 'RC' 'RF' 'SF' 'SH' 'SP' 'SS' 'TO' 'WA'
 'WB' 'WF' 'WH' 'WI' 'WJ' 'WL' 'WO' 'WP' 'YC' 'TRUE_FIR' 'OTHER_HARDWOOD'
 'MAPLE' 'OAK' 'DOUGLAS_FIR' 'SPRUCE' 'CEDAR' 'PONDEROSA_PINE'
 'OTHER_SOFTWOOD' 'LODGEPOLE_PIN

In [12]:
OUTLIERS = '../data/interim/outlier_uuids.csv'
outliers = pd.read_csv(OUTLIERS)
# filter out the height outliers
df = df[~df.UUID.isin(outliers.outlier_uuid)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4073 entries, 0 to 4072
Columns: 107 entries, UUID to MEAS_YR
dtypes: float64(100), int64(2), object(5)
memory usage: 3.4+ MB


In [13]:
df = df.loc[(df.TOPHT > 0)&(df.TOTAL_COVER > 0)&(df.QMD > 0)]
df.loc[df.QMD > 50, 'QMD'] = 50
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3834 entries, 1 to 4072
Columns: 107 entries, UUID to MEAS_YR
dtypes: float64(100), int64(2), object(5)
memory usage: 3.2+ MB


In [14]:
df.to_csv('../data/processed/lidar_structure_training_data_lidar_cover.csv', index=False, header=True)

## Inspect how many samples we have for different years, regions, etc.

In [15]:
df.groupby(by=['YEAR'])[['UUID']].count().rename({'UUID':'count'}, axis=1)

,count
YEAR,
2010.0,487
2011.0,392
2012.0,5
2013.0,574
2014.0,366
2015.0,309
2016.0,762
2017.0,581
2018.0,358


In [16]:
pd.pivot_table(df, 
               values='UUID', 
               aggfunc='count', 
               index=['MEAS_YR'], 
               columns=['YEAR'], 
               fill_value=0)

YEAR,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0
MEAS_YR,,,,,,,,,
2010,487,0,4,0,2,9,0,0,0
2011,0,392,1,0,2,3,0,0,0
2013,0,0,0,574,3,11,14,27,3
2014,0,0,0,0,317,42,92,35,0
2015,0,0,0,0,42,244,112,29,5
2016,0,0,0,0,0,0,544,34,25
2017,0,0,0,0,0,0,0,456,0
2018,0,0,0,0,0,0,0,0,325


In [17]:
ecoreg_counts = df.groupby(by=['ECOREGION3'])[['UUID', 'YEAR', 'PLOT_SIZE_AC']].nunique()
ecoreg_counts

,UUID,YEAR,PLOT_SIZE_AC
ECOREGION3,,,
blue_mountains,85,2,1
cascades,657,6,3
coast_range,1692,9,2
columbia_plateau,7,3,1
eastern_cascades_slopes_and_foothills,198,5,2
klamath_mountains_california_high_north_coast_range,288,6,2
north_cascades,405,5,2
northern_rockies,101,4,1
puget_lowland,116,6,1


## Available features
The different types of predictor variables we can use to predict a forest attribute, including climate, lidar-derived, soil, and satellite imagery.

In [18]:
USE_LIDAR_COLS = ['strat0_return-proportion', 'strat1_return-proportion', 
                  'strat2_return-proportion', 'strat3_return-proportion', 
                  'strat4_return-proportion', 'strat5_return-proportion', 
                  'height_05-percentile',  'height_25-percentile', 
                  'height_50-percentile', 'height_75-percentile',
                  'height_95_percentile', 'cover', 
                  'potential_volume', 'stddev_height', 
                  'surface_volume', 'kurtosis', 'skewness']
USE_LIDAR_COLS = [col.upper() for col in USE_LIDAR_COLS]
df[USE_LIDAR_COLS].describe()

,STRAT0_RETURN-PROPORTION,STRAT1_RETURN-PROPORTION,STRAT2_RETURN-PROPORTION,STRAT3_RETURN-PROPORTION,STRAT4_RETURN-PROPORTION,STRAT5_RETURN-PROPORTION,HEIGHT_05-PERCENTILE,HEIGHT_25-PERCENTILE,HEIGHT_50-PERCENTILE,HEIGHT_75-PERCENTILE,HEIGHT_95_PERCENTILE,COVER,POTENTIAL_VOLUME,STDDEV_HEIGHT,SURFACE_VOLUME,KURTOSIS,SKEWNESS
count,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000
mean,0.160452,0.064758,0.101705,0.209455,0.177241,0.197785,1.259226,9.648884,17.329583,24.133792,32.197671,77.478997,3292.205637,6.416928,1989.334152,4.217114,0.054266
std,0.181026,0.070151,0.113448,0.188833,0.176981,0.257019,5.408052,10.842344,13.567674,15.229965,16.861420,20.973250,1694.542669,4.150131,1365.548481,13.982405,1.382902
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.103300,-3.370587
25%,0.042195,0.020416,0.021888,0.061247,0.027158,0.000000,0.000000,0.390000,5.886250,12.610000,19.723500,68.906629,2041.125000,3.228653,888.140396,1.955420,-0.753138
50%,0.086286,0.041918,0.059361,0.158348,0.126651,0.051950,0.000000,6.290000,16.240000,22.760000,30.107250,85.362201,3117.000000,5.399556,1856.541138,2.602262,-0.102241
75%,0.204105,0.081076,0.139455,0.298456,0.278370,0.363624,0.290000,15.800000,25.600000,33.267499,42.927376,92.714566,4339.500000,8.679241,2837.768494,3.900820,0.648880
max,1.000000,0.626115,0.837302,0.933384,0.866815,1.000000,98.510002,108.769997,116.809998,119.860001,121.559998,100.000000,12180.000000,28.338732,11849.857422,400.029964,18.986166


## Selecting features and targets
This is the first step in determining what features we want to use, and what we want to predict.

In [19]:
X_COLS = USE_LIDAR_COLS + ['ELEVATION', 'LAT', 'LON'] + ['ECOREGION3'] 
Y_COLS = ['COVER', 'TOPHT', 'QMD', 'SDI', 'TCUFT', 'ABOVEGROUND_BIOMASS']

Y_NAMES = [col.upper() for col in Y_COLS]

In [20]:
USE_REGIONS = ['blue_mountains', 'coast_range', 'north_cascades', 'cascades',
               'klamath_mountains_california_high_north_coast_range', 
               'eastern_cascades_slopes_and_foothills', 'northern_rockies',
               'puget_lowland', 'willamette_valley']
display(df.groupby('ECOREGION3')[Y_COLS].mean().round(1).loc[USE_REGIONS])
display(df[Y_COLS].describe())

,TOTAL_COVER,TOPHT,QMD,SDI,TCUFT,ABOVEGROUND_BIOMASS
ECOREGION3,,,,,,
blue_mountains,32.4,59.5,12.5,126.3,1851.8,52.4
coast_range,68.6,107.8,14.7,354.5,9386.4,228.0
north_cascades,63.0,87.3,11.9,303.9,5976.9,159.5
cascades,64.1,102.2,13.3,339.6,7774.7,194.3
klamath_mountains_california_high_north_coast_range,56.2,87.2,10.1,341.7,6567.8,140.3
eastern_cascades_slopes_and_foothills,44.5,76.9,12.0,209.7,3902.4,90.1
northern_rockies,43.4,74.3,11.1,182.0,2740.8,74.0
puget_lowland,68.0,93.3,12.4,296.9,6288.1,184.4
willamette_valley,64.1,111.4,17.4,326.0,9277.6,238.5


,TOTAL_COVER,TOPHT,QMD,SDI,TCUFT,ABOVEGROUND_BIOMASS
count,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000,3834.000000
mean,63.158842,98.537559,13.424433,324.418884,7697.220918,189.444184
std,20.894079,41.156739,7.520646,194.863490,7028.290334,123.966299
min,10.000000,9.000000,1.000000,6.000000,1.000000,6.000000
25%,50.000000,68.000000,8.014735,183.000000,2522.000000,101.000000
50%,67.000000,97.000000,12.228978,303.000000,5831.000000,168.000000
75%,79.000000,126.000000,17.253404,434.750000,10610.750000,250.000000
max,100.000000,256.000000,50.000000,1409.000000,52138.000000,899.000000


In [21]:
df = df.reset_index(drop=True)
X, Y = df[X_COLS], df[Y_COLS]

In [22]:
df[X_COLS].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3834 entries, 0 to 3833
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   STRAT0_RETURN-PROPORTION  3834 non-null   float64
 1   STRAT1_RETURN-PROPORTION  3834 non-null   float64
 2   STRAT2_RETURN-PROPORTION  3834 non-null   float64
 3   STRAT3_RETURN-PROPORTION  3834 non-null   float64
 4   STRAT4_RETURN-PROPORTION  3834 non-null   float64
 5   STRAT5_RETURN-PROPORTION  3834 non-null   float64
 6   HEIGHT_05-PERCENTILE      3834 non-null   float64
 7   HEIGHT_25-PERCENTILE      3834 non-null   float64
 8   HEIGHT_50-PERCENTILE      3834 non-null   float64
 9   HEIGHT_75-PERCENTILE      3834 non-null   float64
 10  HEIGHT_95_PERCENTILE      3834 non-null   float64
 11  COVER                     3834 non-null   float64
 12  POTENTIAL_VOLUME          3834 non-null   float64
 13  STDDEV_HEIGHT             3834 non-null   float64
 14  SURFACE_

## Split datasets by ecoregion
We want to explore model transferability between regions, so we'll train models independently on subsets of the data within a single ecoregion, as well as a model that is trained on all available ecoregions. 

In [23]:
ecoregions = list(np.sort([reg for reg in pd.unique(df.ECOREGION3) if ecoreg_counts.loc[reg]['UUID'] > 20]))

eco_X_idx = [X.loc[X.ECOREGION3 == eco].index.values for eco in ecoregions]

eco_X_dfs = [X.loc[X.ECOREGION3 == eco].drop(['ECOREGION3'], axis=1) for eco in ecoregions]
eco_Y_dfs = [Y.loc[idx] for idx in eco_X_idx]

# append a "global" model that contains data from all ecoregions
ecoregions.append('all')
ecoregion_names = ['_'.join(x.split('_')[0:2]) for x in ecoregions]
eco_X_dfs.append(X.drop(['ECOREGION3'], axis=1))
eco_Y_dfs.append(Y)

ecoregion_display_names = [' '.join(x.upper().split('_')[:2]) for x in ecoregions]

In [24]:
cover_class_bins = [10,40,70,100]
cover_class_labels = ['OPEN', 'MODERATE', 'CLOSED']
height_class_bins = np.arange(0,300,20)
height_class_labels = [f'{x}-{x+20}' for x in height_class_bins[:-1]]
diameter_class_bins = [1, 5, 10, 15, 20, 999]
diameter_class_labels = ['SEED/SAP', 'SMALL', 'MEDIUM', 'LARGE', 'VERY_LARGE']

## Scoring
We'll use Root Mean Square Error to evaluate model performance.

In [25]:
def rmse(obs, pred):
    return np.sqrt((np.square(obs-pred)).mean())

def nrmse(obs, pred):
    return rmse(pred,obs) / obs.mean()

def mae(obs, pred):   
    return abs(pred - obs).mean()

def mape(obs, pred):    
    return abs(pred - obs).mean() / obs.mean()

def bias(obs, pred):   
    return (pred - obs).mean()

def rel_bias(obs, pred):
    return bias(pred,obs) / obs.mean()

def bin_accuracy(obs, pred, bins, fuzzy_tol=0):
    pred_binned = np.digitize(pred, bins)
    obs_binned = np.digitisze(obs, bins)
    diff = abs(pred_binned - obs_binned)
    
    return (diff <= fuzzy_tol).sum() / len(diff)

def confidence_interval_half(X, confidence=0.95):
    n = len(X)
    se = stats.sem(X)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return h

## Fit some models
For each type of model, we'll employ cross-validation to tune model hyperparameters, generating a tuned model for each ecoregion as well as a tuned model using all training data. 

In [26]:
MODELS = {
    'ElasticNet': ElasticNet(),
    'Lasso': Lasso(), 
    'KNeighborsRegressor': KNeighborsRegressor(n_jobs=-1),
    'RandomForestRegressor': RandomForestRegressor(n_jobs=-1), 
    'HistGradientBoostingRegressor': HistGradientBoostingRegressor(), 
}

FIT_PARAMS = {
    'ElasticNet': {
        'alpha': np.logspace(-4,2,7),
        'l1_ratio': np.arange(0.0, 1.0, 0.1),
    },
    'Lasso': {
        'alpha': np.logspace(-4,2,7),
    },
    'KNeighborsRegressor': {
        'n_neighbors': [1,2,3,4,5,10,20],
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski', 'manhattan']
    },
    'RandomForestRegressor': {
        'n_estimators': [100, 500, 1000],
        'max_features': ['sqrt', None],
        'max_depth': [5, 20, None],
        'max_samples': [0.5, None]
    },
    'HistGradientBoostingRegressor': {
        'max_iter': [50, 100, 200],
        'min_samples_leaf': [5, 10, 20],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1],
    },
}

In [27]:
NUM_OUTER_FOLDS = 5
NUM_INNER_FOLDS = 3
SCORE_FUNCS = [rmse, nrmse, mae, mape, bias, rel_bias]
score_names = [func.__name__ for func in SCORE_FUNCS]

In [28]:
def build_insider_results_dictionary(regions, model_names, num_outer_folds, score_funcs, target_vars):
    results = {}
    for region in regions:
        results[region] = {}
        for model_name in model_names:
            results[region][model_name] = {}
            results[region][model_name]['fitted_model'] = None
            results[region][model_name]['best_params'] = None
            results[region][model_name]['cv_results'] = {}
            for fold_idx in range(num_outer_folds):  # results from each outer loop of nested CV
                fold_num = fold_idx + 1
                results[region][model_name]['cv_results'][fold_num] = {}
                results[region][model_name]['cv_results'][fold_num]['best_params'] = None 
                results[region][model_name]['cv_results'][fold_num]['predict_time'] = None
                for score_func in score_funcs:
                    score_func_name = score_func.__name__
                    results[region][model_name]['cv_results'][fold_num][score_func_name] = {
                        y: None for y in target_vars
                    }
    return results

def build_global_results_dictionary(regions, model_names, num_outer_folds, score_funcs, target_vars):
    results = {}
    for model_name in model_names:
        results[model_name] = {}
        results[model_name]['fitted_model'] = None
        results[model_name]['best_params'] = None
        results[model_name]['cv_results'] = {}
        for fold_idx in range(num_outer_folds):  # results from each outer loop of nested CV
            fold_num = fold_idx + 1
            results[model_name]['cv_results'][fold_num] = {}
            results[model_name]['cv_results'][fold_num]['best_params'] = None 
            results[model_name]['cv_results'][fold_num]['predict_time'] = None
            for region in regions:
                results[model_name]['cv_results'][fold_num][region] = {}
                for score_func in score_funcs:
                    score_func_name = score_func.__name__
                    results[model_name]['cv_results'][fold_num][region][score_func_name] = {
                        y: None for y in target_vars
                    }
    return results

def build_outsider_results_dictionary(regions, model_names, score_funcs, target_vars):
    results = {}
    for region in regions:
        results[region] = {}
        for model_name in model_names:
            results[region][model_name] = {}
            results[region][model_name]['fitted_model'] = None
            results[region][model_name]['best_params'] = None
            results[region][model_name]['predict_time'] = None
            for score_func in score_funcs:
                score_func_name = score_func.__name__
                results[region][model_name][score_func_name] = {
                    y: None for y in target_vars
                }
    return results

def build_visiting_insider_results_dictionary(regions, model_names, score_funcs, target_vars):
    results = {}
    for target_region in regions:
        results[target_region] = {}
        for train_region in [r for r in regions if r != target_region]:
            results[target_region][train_region] = {}
            for model_name in model_names:
                results[target_region][train_region][model_name] = {}
                for score_func in score_funcs:
                    score_func_name = score_func.__name__
                    results[target_region][train_region][model_name][score_func_name] = {
                        y: None for y in target_vars
                    }
    return results

In [42]:
def tune_insider_model(model_name, num_outer_folds=NUM_OUTER_FOLDS, num_inner_folds=NUM_INNER_FOLDS):
    print(model_name)
    print('-'*len(model_name))
    model = MODELS[model_name]
    fit_params = FIT_PARAMS[model_name]
    train_regions = [x for x in ecoregions if x.upper() != 'ALL']
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RegressorChain(base_estimator=model)),
    ])
    search_params = {f'model__base_estimator__{key}': value for key, value in fit_params.items()}
    
    cv_outer = GroupKFold(num_outer_folds)
    cv_inner = GroupKFold(num_inner_folds)
    
    for i, ecoregion in enumerate(train_regions):
        ecoregion_name = ecoregion_display_names[i]
        print(f'Starting on {ecoregion_name}', end='... ')
        X = eco_X_dfs[i]
        Y = eco_Y_dfs[i]
        outer_groups = df.loc[X.index, 'UUID'].values
        
        outer_fold_num = 1
        for train_ix, test_ix in cv_outer.split(X, groups=outer_groups):
            X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
            Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
            inner_groups = df.loc[X_train.index, 'UUID'].values
            
            inner_search = GridSearchCV(pipe, search_params, 
                                        scoring='neg_mean_squared_error', 
                                        n_jobs=-1, cv=cv_inner, refit=True)
            
            inner_result = inner_search.fit(X_train, Y_train, groups=inner_groups)
            insider_results[ecoregion][model_name]['cv_results'][outer_fold_num]['best_params'] = inner_result.best_params_
            
            inner_best_model = inner_result.best_estimator_
            start_time = time.time()
            Y_pred = inner_best_model.predict(X_test)
            end_time = time.time()
            total_predict_time = end_time - start_time
            avg_predict_time = total_predict_time / len(X_test)
            insider_results[ecoregion][model_name]['cv_results'][outer_fold_num]['predict_time'] = avg_predict_time
            
            for score_func in SCORE_FUNCS:
                score_func_name = score_func.__name__
                scores = score_func(Y_test, Y_pred)
                for y_var in scores.index:
                    insider_results[ecoregion][model_name]['cv_results'][outer_fold_num][score_func_name][y_var] = scores.loc[y_var]
                    
            print(outer_fold_num, end='... ')
            outer_fold_num += 1
        print('Done scoring. Now fitting a final model', end='... ')
        
        # done with scoring of models, now time to tune a model using the whole dataset
        outer_search = GridSearchCV(pipe, search_params, 
                                    scoring='neg_mean_squared_error', 
                                    n_jobs=-1, cv=cv_outer, refit=True)
        outer_result = outer_search.fit(X, Y, groups=outer_groups)
        
        # now fit on the entire dataset, not just training set
        model = outer_result.best_estimator_
        model.set_params(**outer_result.best_params_)
        X = df.loc[df.ECOREGION3 == ecoregion, X_COLS].drop(['ECOREGION3'], axis=1)
        y = df.loc[df.ECOREGION3 == ecoregion, Y_COLS]
        model.fit(X, y)

        eco_name = '_'.join(ecoregion.split('_')[:2])
        outfile = f'{eco_name}-lidar-{model_name}-chained.pkl'
        outpath = os.path.join('../models/structure_models', outfile)
        with open(outpath, 'wb') as file:
            pickle.dump(model, file)
        
        insider_results[ecoregion][model_name]['fitted_model'] = model
        insider_results[ecoregion][model_name]['best_params'] = outer_result.best_params_
        
        cv_results_dict = {ecoregion: insider_results[ecoregion][model_name]['cv_results'] for ecoregion in train_regions}
        print('All done.')
    
    return cv_results_dict

def tune_outsider_model(model_name, num_folds=5):
    print(model_name)
    print('-'*len(model_name))
    model = MODELS[model_name]
    fit_params = FIT_PARAMS[model_name]
    train_regions = [x for x in ecoregions if x.upper() != 'ALL']
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RegressorChain(base_estimator=model)),
    ])
    search_params = {f'model__base_estimator__{key}': value for key, value in fit_params.items()}
    
    groupkfold = GroupKFold(num_folds)
    
    for i, ecoregion in enumerate(train_regions):
        ecoregion_name = ecoregion_display_names[i]
        print(f'Starting on {ecoregion_name}', end='... ')
        X_train = X.loc[X.ECOREGION3 != ecoregion].drop('ECOREGION3', axis=1)
        Y_train = Y.loc[X_train.index]
        X_test = X.loc[X.ECOREGION3 == ecoregion].drop('ECOREGION3', axis=1)
        Y_test = Y.loc[X_test.index]
        groups = df.loc[X_train.index]['ECOREGION3'].values
        
        search = GridSearchCV(pipe, search_params, 
                              scoring='neg_mean_squared_error',
                              n_jobs=-1, cv=groupkfold, refit=True)
            
        result = search.fit(X_train, Y_train, groups=groups)
        print('Done fitting, now scoring', end='... ')
        outsider_results[ecoregion][model_name]['best_params'] = result.best_params_
        outsider_results[ecoregion][model_name]['fitted_model'] = result.best_estimator_
        
        best_model = result.best_estimator_       
        start_time = time.time()
        Y_pred = best_model.predict(X_test)
        end_time = time.time()
        total_predict_time = end_time - start_time
        avg_predict_time = total_predict_time / len(X_test)
        outsider_results[ecoregion][model_name]['predict_time'] = avg_predict_time
            
        for score_func in SCORE_FUNCS:
            score_func_name = score_func.__name__
            scores = score_func(Y_test, Y_pred)
            for y_var in scores.index:
                outsider_results[ecoregion][model_name][score_func_name][y_var] = scores.loc[y_var]
        
        results_dict = {ecoregion: outsider_results[ecoregion][model_name] for ecoregion in train_regions}
        print('All done.')
    
    return results_dict

def tune_global_model(model_name, num_outer_folds=NUM_OUTER_FOLDS, num_inner_folds=NUM_INNER_FOLDS):
    print(model_name)
    print('-'*len(model_name))
    print(f'Scoring with {NUM_OUTER_FOLDS} folds... ', end='')
    model = MODELS[model_name]
    fit_params = FIT_PARAMS[model_name]
    test_regions = [x for x in ecoregions if x.upper() != 'ALL']
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RegressorChain(base_estimator=model)),
    ])
    search_params = {f'model__base_estimator__{key}': value for key, value in fit_params.items()}
    
    cv_outer = GroupKFold(num_outer_folds)
    cv_inner = GroupKFold(num_inner_folds)
    
    X = df[X_COLS].drop('ECOREGION3', axis=1)
    Y = df[Y_COLS]
    outer_groups = df['UUID'].values
        
    outer_fold_num = 1
    for train_ix, test_ix in cv_outer.split(X, groups=outer_groups):
        X_train, X_test = X.loc[train_ix], X.loc[test_ix]
        Y_train, Y_test = Y.loc[train_ix], Y.loc[test_ix]
        inner_groups = df.loc[train_ix, 'UUID'].values

        inner_search = GridSearchCV(pipe, search_params, 
                                    scoring='neg_mean_squared_error', 
                                    n_jobs=-1, cv=cv_inner, refit=True)

        inner_result = inner_search.fit(X_train, Y_train, groups=inner_groups)
        global_results[model_name]['cv_results'][outer_fold_num]['best_params'] = inner_result.best_params_

        inner_best_model = inner_result.best_estimator_
        start_time = time.time()
        Y_pred = inner_best_model.predict(X_test)
        end_time = time.time()
        total_predict_time = end_time - start_time
        avg_predict_time = total_predict_time / len(X_test)
        global_results[model_name]['cv_results'][outer_fold_num]['predict_time'] = avg_predict_time

        for ecoregion in test_regions:
            region_mask = (df.loc[test_ix, 'ECOREGION3'] == ecoregion).values
            regional_X_test = X_test.loc[test_ix[region_mask]]
            regional_Y_test = Y_test.loc[test_ix[region_mask]]
            regional_Y_pred = inner_best_model.predict(regional_X_test)
            
            for score_func in SCORE_FUNCS:
                score_func_name = score_func.__name__
                scores = score_func(regional_Y_test, regional_Y_pred)
                for y_var in scores.index:
                    global_results[model_name]['cv_results'][outer_fold_num][ecoregion][score_func_name][y_var] = scores.loc[y_var]

        print(outer_fold_num, end='... ')
        outer_fold_num += 1
    
    print('Done scoring. Now fitting a final model', end='... ')
        
    # done with scoring of models, now time to tune a model using the whole dataset
    outer_search = GridSearchCV(pipe, search_params, 
                                scoring='neg_mean_squared_error', 
                                n_jobs=-1, cv=cv_outer, refit=True)
    outer_result = outer_search.fit(X, Y, groups=outer_groups)

     # now fit on the entire dataset, not just training set
    model = outer_result.best_estimator_
    model.set_params(**outer_result.best_params_)
    X = df[X_COLS].drop(['ECOREGION3'], axis=1)
    y = df[Y_COLS]
    model.fit(X, y)

    outfile = f'global-lidar-{model_name}-chained.pkl'
    outpath = os.path.join('../models/structure_models', outfile)
    with open(outpath, 'wb') as file:
        pickle.dump(model, file)
    print('All done.')

    global_results[model_name]['fitted_model'] = outer_result.best_estimator_
    global_results[model_name]['best_params'] = outer_result.best_params_

    results_dict = global_results[model_name]
    
    return results_dict

In [30]:
def parse_global_results(results):
    data = []
    for fold in range(NUM_OUTER_FOLDS):
        for ecoregion in ecoregions[:-1]:
            for target in Y_COLS:
                for score_name in score_names:
                    data.append((fold+1, ecoregion, target, score_name, results['cv_results'][fold+1][ecoregion][score_name][target]))
    return pd.DataFrame(data, columns=['cv_fold', 'ecoregion', 'target', 'metric', 'score'])

## Fit Global Models
These models get to see data from every ecoregion during training and tuning.

In [31]:
global_results = build_global_results_dictionary(ecoregions[:-1], MODELS.keys(), NUM_OUTER_FOLDS, SCORE_FUNCS, Y_COLS)

In [36]:
os.environ["PYTHONWARNINGS"] = "ignore"
elastic_global = tune_global_model('ElasticNet')
os.environ["PYTHONWARNINGS"] = ""
lasso_global = tune_global_model('Lasso')
knn_global = tune_global_model('KNeighborsRegressor')
rf_global = tune_global_model('RandomForestRegressor')
gbm_global = tune_global_model('HistGradientBoostingRegressor')

ElasticNet
----------
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Lasso
-----
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
KNeighborsRegressor
-------------------
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
RandomForestRegressor
---------------------
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
HistGradientBoostingRegressor
-----------------------------
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.


In [37]:
RESULTS_TO_CONCAT = [elastic_global, lasso_global, knn_global, rf_global, gbm_global]
NAMES = ['ElasticNet', 'Lasso', 'kNN', 'RF', 'GBM']
dfs_to_concat = []
for res, name in zip(RESULTS_TO_CONCAT, NAMES):
    tmp_df = parse_global_results(res)
    tmp_df['model'] = name
    dfs_to_concat.append(tmp_df)
all_global_results = pd.concat(dfs_to_concat, axis=0, ignore_index=True)
all_global_results['ecoregion'] = all_global_results['ecoregion'].apply(lambda x: ' '.join(x.title().replace('_',' ').split()[:2]))
all_global_results.columns = [col.upper() for col in all_global_results.columns]
# all_global_results = all_global_results.rename({'SCORE': 'MAE'}, axis=1)
all_global_results.head()

,CV_FOLD,ECOREGION,TARGET,METRIC,SCORE,MODEL
0,1,Blue Mountains,TOTAL_COVER,rmse,7.528853,ElasticNet
1,1,Blue Mountains,TOTAL_COVER,nrmse,0.248204,ElasticNet
2,1,Blue Mountains,TOTAL_COVER,mae,5.490831,ElasticNet
3,1,Blue Mountains,TOTAL_COVER,mape,0.181016,ElasticNet
4,1,Blue Mountains,TOTAL_COVER,bias,-2.943946,ElasticNet


In [38]:
all_global_results.to_csv('../data/processed/nestedcv_chained_global_results_lidar_structure.csv', header=True, index=False)

## Fit Insider Models
These models are trained with observations from a single ecoregion.

In [39]:
insider_results = build_insider_results_dictionary(ecoregions[:-1], MODELS.keys(), 5, SCORE_FUNCS, Y_COLS)

In [40]:
def parse_insider_results(results):
    data = []
    for ecoregion in ecoregions[:-1]:
        for fold_num in results[ecoregion].keys():
            for target in Y_COLS:
                for score_name in score_names:
                    data.append((fold_num, ecoregion, target, score_name, results[ecoregion][fold_num][score_name][target]))
    return pd.DataFrame(data, columns=['cv_fold', 'ecoregion', 'target', 'metric', 'score'])

In [43]:
os.environ["PYTHONWARNINGS"] = "ignore"
elastic_insider = tune_insider_model('ElasticNet')
os.environ["PYTHONWARNINGS"] = ""
lasso_insider = tune_insider_model('Lasso')
knn_insider = tune_insider_model('KNeighborsRegressor')
rf_insider = tune_insider_model('RandomForestRegressor')
gbm_insider = tune_insider_model('HistGradientBoostingRegressor')

ElasticNet
----------
Starting on BLUE MOUNTAINS... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on CASCADES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on COAST RANGE... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on EASTERN CASCADES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on KLAMATH MOUNTAINS... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on NORTH CASCADES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on NORTHERN ROCKIES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on PUGET LOWLAND... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on WILLAMETTE VALLEY... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Lasso


In [44]:
RESULTS_TO_CONCAT = [elastic_insider, lasso_insider, knn_insider, rf_insider, gbm_insider]
NAMES = ['ElasticNet', 'Lasso', 'kNN', 'RF', 'GBM']
dfs_to_concat = []
for res, name in zip(RESULTS_TO_CONCAT, NAMES):
    tmp_df = parse_insider_results(res)
    tmp_df['model'] = name
    dfs_to_concat.append(tmp_df)
all_insider_results = pd.concat(dfs_to_concat, axis=0, ignore_index=True)
all_insider_results['ecoregion'] = all_insider_results['ecoregion'].apply(lambda x: ' '.join(x.title().replace('_',' ').split()[:2]))
all_insider_results.columns = [col.upper() for col in all_insider_results.columns]
all_insider_results.head()

,CV_FOLD,ECOREGION,TARGET,METRIC,SCORE,MODEL
0,1,Blue Mountains,TOTAL_COVER,rmse,6.844798,ElasticNet
1,1,Blue Mountains,TOTAL_COVER,nrmse,0.201666,ElasticNet
2,1,Blue Mountains,TOTAL_COVER,mae,5.643110,ElasticNet
3,1,Blue Mountains,TOTAL_COVER,mape,0.166261,ElasticNet
4,1,Blue Mountains,TOTAL_COVER,bias,-0.276478,ElasticNet


In [45]:
all_insider_results.to_csv('../data/processed/nestedcv_chained_insider_results_lidar_structure.csv', header=True, index=False)

## Use Trained Insider Models to Score Visiting Insider Models
These models are trained on a single region, and scored on other regions they've never seen before. 

In [46]:
visitor_results = build_visiting_insider_results_dictionary(ecoregions[:-1], MODELS.keys(), SCORE_FUNCS, Y_COLS)

In [47]:
visitor_results = []
for target_region in ecoregions[:-1]:
    for train_region in [r for r in ecoregions[:-1] if r != target_region]:
        for model_name in MODELS.keys():
            model = insider_results[train_region][model_name]['fitted_model']
            targ_idx = X.loc[X.ECOREGION3 == target_region].index.values
            targ_X = X.loc[targ_idx].drop(['ECOREGION3'], axis=1)
            pred = model.predict(targ_X)
            obs = Y.loc[targ_idx]
            for score_func in SCORE_FUNCS:
                score_func_name = score_func.__name__
                scores = score_func(obs, pred)
                for y, score in scores.iteritems():
                    visitor_results.append(
                        (' '.join(target_region.title().replace('_',' ').split()),
                         ' '.join(train_region.title().replace('_',' ').split()),
                         model_name, score_func_name, y, score))
visitor_df = pd.DataFrame(visitor_results, 
                          columns = ['TARGET_ECOREGION', 'TRAIN_ECOREGION', 
                                     'MODEL', 'METRIC', 'TARGET', 'SCORE'])
visitor_df.head()

,TARGET_ECOREGION,TRAIN_ECOREGION,MODEL,METRIC,TARGET,SCORE
0,Blue Mountains,Cascades,ElasticNet,rmse,TOTAL_COVER,16.037629
1,Blue Mountains,Cascades,ElasticNet,rmse,TOPHT,27.830946
2,Blue Mountains,Cascades,ElasticNet,rmse,QMD,6.141004
3,Blue Mountains,Cascades,ElasticNet,rmse,SDI,245.765012
4,Blue Mountains,Cascades,ElasticNet,rmse,TCUFT,3564.342851


In [48]:
visitor_df.to_csv('../data/processed/nestedcv_chained_visitor_results_lidar_structure.csv', 
                  header=True, index=False)

## Fit Outsider Models
These models have data from the ecoregion they're tested on held out during training.

In [49]:
outsider_results = build_outsider_results_dictionary(ecoregions[:-1], MODELS.keys(), SCORE_FUNCS, Y_COLS)

In [50]:
def parse_outsider_results(results):
    data = []
    for ecoregion in ecoregions[:-1]:
        for target in Y_COLS:
            for score_name in score_names:
                data.append((np.nan, ecoregion, target, score_name, results[ecoregion][score_name][target]))
    return pd.DataFrame(data, columns=['cv_fold', 'ecoregion', 'target', 'metric', 'score'])

In [51]:
os.environ["PYTHONWARNINGS"] = "ignore"
elastic_outsider = tune_outsider_model('ElasticNet')
os.environ["PYTHONWARNINGS"] = "ignore"
lasso_outsider = tune_outsider_model('Lasso')
knn_outsider = tune_outsider_model('KNeighborsRegressor')
rf_outsider = tune_outsider_model('RandomForestRegressor')
gbm_outsider = tune_outsider_model('HistGradientBoostingRegressor')

ElasticNet
----------
Starting on BLUE MOUNTAINS... Done fitting, now scoring... All done.
Starting on CASCADES... Done fitting, now scoring... All done.
Starting on COAST RANGE... Done fitting, now scoring... All done.
Starting on EASTERN CASCADES... Done fitting, now scoring... All done.
Starting on KLAMATH MOUNTAINS... Done fitting, now scoring... All done.
Starting on NORTH CASCADES... Done fitting, now scoring... All done.
Starting on NORTHERN ROCKIES... Done fitting, now scoring... All done.
Starting on PUGET LOWLAND... Done fitting, now scoring... All done.
Starting on WILLAMETTE VALLEY... Done fitting, now scoring... All done.
Lasso
-----
Starting on BLUE MOUNTAINS... Done fitting, now scoring... All done.
Starting on CASCADES... Done fitting, now scoring... All done.
Starting on COAST RANGE... Done fitting, now scoring... All done.
Starting on EASTERN CASCADES... Done fitting, now scoring... All done.
Starting on KLAMATH MOUNTAINS... Done fitting, now scoring... All done.
Star

In [52]:
RESULTS_TO_CONCAT = [elastic_outsider, lasso_outsider, knn_outsider, rf_outsider, gbm_outsider]
NAMES = ['ElasticNet', 'Lasso', 'kNN', 'RF', 'GBM']
dfs_to_concat = []
for res, name in zip(RESULTS_TO_CONCAT, NAMES):
    tmp_df = parse_outsider_results(res)
    tmp_df['model'] = name
    dfs_to_concat.append(tmp_df)
all_outsider_results = pd.concat(dfs_to_concat, axis=0, ignore_index=True)
all_outsider_results['ecoregion'] = all_outsider_results['ecoregion'].apply(lambda x: ' '.join(x.title().replace('_',' ').split()[:2]))
all_outsider_results.columns = [col.upper() for col in all_outsider_results.columns]
all_outsider_results.head()

,CV_FOLD,ECOREGION,TARGET,METRIC,SCORE,MODEL
0,NaN,Blue Mountains,TOTAL_COVER,rmse,7.594567,ElasticNet
1,NaN,Blue Mountains,TOTAL_COVER,nrmse,0.234315,ElasticNet
2,NaN,Blue Mountains,TOTAL_COVER,mae,5.788852,ElasticNet
3,NaN,Blue Mountains,TOTAL_COVER,mape,0.178603,ElasticNet
4,NaN,Blue Mountains,TOTAL_COVER,bias,-2.248044,ElasticNet


In [53]:
all_outsider_results.to_csv('../data/processed/nestedcv_chained_outsider_results_lidar_structure.csv', header=True, index=False)